In [3]:
import cv2
import mediapipe as mp
from mediapipe.tasks.python.vision.gesture_recognizer import GestureRecognizerResult
import numpy as np
import math
import time

class PaintApp:
    def __init__(self):
        # Initialize MediaPipe
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(
            static_image_mode=False, 
            max_num_hands=1, 
            min_detection_confidence=0.5
        )
        self.mp_drawing = mp.solutions.drawing_utils
        
        # Initialize variables
        self.colors = [(0, 0, 255), (0, 255, 0), (255, 0, 0), (0, 255, 255)]
        self.current_color = self.colors[0]
        self.brush_size = 10
        self.menu_width = 100
        self.drawing = False
        self.canvas = None
        self.dominant_hand = None
        self.calibrating = True
        
        # Initialize gesture recognizer
        self.model_path = './gesture_recognizer.task'
        self.setup_gesture_recognizer()
        
    def setup_gesture_recognizer(self):
        base_options = mp.tasks.BaseOptions(model_asset_path=self.model_path)
        options = mp.tasks.vision.GestureRecognizerOptions(
            base_options=base_options,
            running_mode=mp.tasks.vision.RunningMode.LIVE_STREAM,
            result_callback=self.gesture_callback
        )
        self.recognizer = mp.tasks.vision.GestureRecognizer.create_from_options(options)
    
    def gesture_callback(self, result: GestureRecognizerResult, output_image: mp.Image, timestamp_ms: int):
            print(result)
            if self.calibrating:
                if result.gestures:
                    print('go to dectect_dominant_hand')
                    self.dominant_hand = self.detect_dominant_hand(result)
                    if self.dominant_hand:
                        self.calibrating = False
                        print('dominant hand set to ', self.dominant_hand)

    def detect_dominant_hand(self, result):
        # Detecta si la mano es izquierda o derecha.
        print(0)
        if result.hand_landmarks and result.handedness and result.gestures:
            # Check each hand
            print('check each hand')
            for hand_idx in range(len(result.hand_landmarks)):
                print('1')
                hand_label = result.handedness[hand_idx][0].category_name
                gesture_name = result.gestures[hand_idx][0].category_name
                print('2')
                
                if gesture_name == "Open_Palm":
                    print('3')
                    return hand_label
                
        # If no hand with Open_Palm was found
        return None
    
    def count_visible_fingers(self, landmarks):
        # Implement your finger counting logic here
        return 1  # placeholder
    
    def calculate_brush_size(self, landmarks):
        # Implement your brush size calculation logic here
        return 10  # placeholder
    
    def draw_menu(self, frame):
        for i, color in enumerate(self.colors):
            cv2.circle(frame, (50, 100 + i * 100), 30, color, -1)
    
    def run(self):
        cap = cv2.VideoCapture(0)
        
        try:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                if self.canvas is None:
                    self.canvas = np.zeros_like(frame)

                current_timestamp_ms = int(time.time() * 1000)
                frame = cv2.flip(frame, 1)
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                res_hands = self.hands.process(rgb_frame)

                mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)
                frame_height, frame_width, _ = frame.shape

                if self.calibrating:
                    cv2.putText(frame, "Muestra tu mano dominante", (10, 50), 
                              cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
                    self.recognizer.recognize_async(mp_image, current_timestamp_ms)
                else:
                    self.process_hand_tracking(frame, res_hands, frame_width, frame_height)

                frame = cv2.addWeighted(frame, 0.5, self.canvas, 0.5, 0)
                self.draw_menu(frame)
                cv2.imshow("Paint Interactivo", frame)

                key = cv2.waitKey(1)
                if key & 0xFF == 27:
                    break
                elif key & 0xFF == ord('d'):
                    self.drawing = not self.drawing

        finally:
            # Cleanup
            self.cleanup(cap)
    
    def process_hand_tracking(self, frame, res_hands, frame_width, frame_height):
        if res_hands.multi_handedness and res_hands.multi_hand_landmarks:
            for i, hand_landmarks in enumerate(res_hands.multi_hand_landmarks):
                handedness = res_hands.multi_handedness[i].classification[0].label
                if handedness != self.dominant_hand:
                    continue

                index_finger = hand_landmarks.landmark[self.mp_hands.HandLandmark.INDEX_FINGER_TIP]
                x, y = int(index_finger.x * frame_width), int(index_finger.y * frame_height)
                
                visible_fingers = self.count_visible_fingers(hand_landmarks)
                self.update_drawing_state(visible_fingers, hand_landmarks)
                
                self.handle_menu_interaction(frame, hand_landmarks, x, y, frame_width)
                
                if self.drawing:
                    cv2.circle(self.canvas, (x, y), self.brush_size, self.current_color, -1)
                cv2.circle(frame, (x, y), 10, self.current_color, -1)
    
    def update_drawing_state(self, visible_fingers, landmarks):
        if visible_fingers == 0:
            self.drawing = False
        else:
            self.drawing = True
            if visible_fingers == 1:
                self.brush_size = 10
            elif visible_fingers >= 2:
                self.brush_size = self.calculate_brush_size(landmarks)
    
    def handle_menu_interaction(self, frame, landmarks, x, y, frame_width):
        wrist_x = int(landmarks.landmark[self.mp_hands.HandLandmark.WRIST].x * frame_width)
        if wrist_x < self.menu_width:
            for i, color in enumerate(self.colors):
                cx, cy = 50, 100 + i * 100
                if math.hypot(x - cx, y - cy) < 30:
                    self.current_color = color
    
    def cleanup(self, cap):
        self.recognizer.close()
        self.hands.close()
        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    app = PaintApp()
    app.run()

I0000 00:00:1736088286.348892  194869 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1736088286.352130  209031 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.9-0ubuntu0.2), renderer: Mesa Intel(R) UHD Graphics 620 (WHL GT2)
I0000 00:00:1736088286.357884  194869 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1736088286.362347  209034 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.9-0ubuntu0.2), renderer: Mesa Intel(R) UHD Graphics 620 (WHL GT2)
W0000 00:00:1736088286.364132  194869 gesture_recognizer_graph.cc:129] Hand Gesture Recognizer contains CPU only ops. Sets HandGestureRecognizerGraph acceleration to Xnnpack.
I0000 00:00:1736088286.369186  194869 hand_gesture_recognizer_graph.cc:250] Custom gesture classifier is not defined.
W0000 00:00:1736088286.462428  209021 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support f

GestureRecognizerResult(gestures=[], handedness=[], hand_landmarks=[], hand_world_landmarks=[])
GestureRecognizerResult(gestures=[], handedness=[], hand_landmarks=[], hand_world_landmarks=[])
GestureRecognizerResult(gestures=[], handedness=[], hand_landmarks=[], hand_world_landmarks=[])
GestureRecognizerResult(gestures=[], handedness=[], hand_landmarks=[], hand_world_landmarks=[])
GestureRecognizerResult(gestures=[], handedness=[], hand_landmarks=[], hand_world_landmarks=[])
GestureRecognizerResult(gestures=[], handedness=[], hand_landmarks=[], hand_world_landmarks=[])
GestureRecognizerResult(gestures=[], handedness=[], hand_landmarks=[], hand_world_landmarks=[])
GestureRecognizerResult(gestures=[], handedness=[], hand_landmarks=[], hand_world_landmarks=[])
GestureRecognizerResult(gestures=[], handedness=[], hand_landmarks=[], hand_world_landmarks=[])
GestureRecognizerResult(gestures=[], handedness=[], hand_landmarks=[], hand_world_landmarks=[])
GestureRecognizerResult(gestures=[], han

In [4]:
''

''